# Data Module


In [ ]:
# default_exp datasets

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [ ]:
# export
from cfnet.import_essentials import *
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OneHotEncoder
from torch.utils.data import Dataset, DataLoader


In [ ]:
# export
class NumpyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        assert self.X.shape[0] == self.y.shape[0]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
# export
# copy from https://jax.readthedocs.io/en/latest/notebooks/Neural_Network_and_Data_Loading.html#data-loading-with-pytorch
def _numpy_collate(batch):
  if isinstance(batch[0], np.ndarray):
    return np.stack(batch)
  elif isinstance(batch[0], (tuple,list)):
    transposed = zip(*batch)
    return [_numpy_collate(samples) for samples in transposed]
  else:
    return np.array(batch)

class NumpyLoader(DataLoader):
  def __init__(self, dataset, batch_size=1,
                shuffle=False, sampler=None,
                batch_sampler=None, num_workers=0,
                pin_memory=False, drop_last=False,
                timeout=0, worker_init_fn=None):
    super(self.__class__, self).__init__(dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        sampler=sampler,
        batch_sampler=batch_sampler,
        num_workers=num_workers,
        collate_fn=_numpy_collate,
        pin_memory=pin_memory,
        drop_last=drop_last,
        timeout=timeout,
        worker_init_fn=worker_init_fn)

In [ ]:
# export
class DataModuleConfigs(BaseParser):
    batch_size: int
    discret_cols: List[str] = []
    continous_cols: List[str] = []
    normalizer: Optional[Any] = None
    encoder: Optional[Any] = None
    sample_frac: Optional[float] = None


In [ ]:
# export
class TabularDataModule:
    discret_cols: List[str] = []
    continous_cols: List[str] = []
    normalizer: Optional[Any] = None
    encoder: Optional[OneHotEncoder] = None
    data: Optional[pd.DataFrame] = None
    sample_frac: Optional[float] = None
    batch_size: int = 128

    def __init__(self, data_configs: Dict):
        # read data
        self.data = pd.read_csv(Path(data_configs['data_dir']))
        # update configs
        self._update_configs(data_configs)
        # update cat_idx
        self.cat_idx = len(self.continous_cols)
        # prepare data
        self.prepare_data()

    def _update_configs(self, configs):
        for k, v in configs.items():
            setattr(self, k, v)

    def prepare_data(self):
        def split_x_and_y(data: pd.DataFrame):
            X = data[data.columns[:-1]]
            y = data[[data.columns[-1]]]
            return X, y

        X, y = split_x_and_y(self.data)

        # preprocessing
        if self.normalizer:
            X_cont = self.normalizer.transform(X[self.continous_cols])
        else:
            self.normalizer = MinMaxScaler()
            X_cont = self.normalizer.fit_transform(
                X[self.continous_cols]) if self.continous_cols else np.array([[] for _ in range(len(X))])

        if self.encoder:
            X_cat = self.encoder.transform(X[self.discret_cols])
        else:
            self.encoder = OneHotEncoder(sparse=False)
            X_cat = self.encoder.fit_transform(
                X[self.discret_cols]) if self.discret_cols else np.array([[] for _ in range(len(X))])
        X = np.concatenate((X_cont, X_cat), axis=1)
        # get categorical arrays
        self.cat_arrays = self.encoder.categories_ if self.discret_cols else []

        # prepare train & test
        train_test_tuple = train_test_split(X, y.to_numpy(), shuffle=False)
        train_X, test_X, train_y, test_y = map(lambda x: x.astype(jnp.float32), train_test_tuple)
        if self.sample_frac:
            train_size = int(len(train_X) * self.sample_frac)
            train_X, train_y = train_X[:train_size], train_y[:train_size]
        self.train_dataset = NumpyDataset(train_X, train_y)
        self.val_dataset = NumpyDataset(test_X, test_y)
        self.test_dataset = self.val_dataset

    def train_dataloader(self):
        return NumpyLoader(self.train_dataset, batch_size=self.batch_size,
                          pin_memory=True, shuffle=True, num_workers=0)

    def val_dataloader(self):
        return NumpyLoader(self.val_dataset, batch_size=self.batch_size * 4,
                          pin_memory=True, shuffle=False, num_workers=0)

    def test_dataloader(self):
        return NumpyLoader(self.val_dataset, batch_size=self.batch_size,
                          pin_memory=True, shuffle=False, num_workers=0)

    def get_sample_X(self, frac: Optional[float]=None):
        if frac is None:
            frac = 0.1
        train_X, train_y = self.train_dataset[:]
        train_size = int(len(train_X) * frac)
        return train_X[:train_size]

In [ ]:
data_configs = {
    "data_dir": "assets/data/s_adult.csv",
    "data_name": "adult",
    "batch_size": 256,
    'sample_frac': 0.1,
    "continous_cols": [
        "age",
        "hours_per_week"
    ],
    "discret_cols": [
        "workclass",
        "education",
        "marital_status",
        "occupation",
        "race",
        "gender"
    ],
}

In [ ]:
dm = TabularDataModule(data_configs)
dataloader = dm.train_dataloader()
x, y = next(iter(dataloader))
assert x.shape[0] == 256
assert x.shape[1] == 29
assert dm.sample_frac == 0.1

In [ ]:
dm.train_dataset[:][1]

array([[1.],
       [0.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)